Usando los datos del ejercicio anterior y de acuerdo a la ley de Zipf, calcule la cantidad de palabras que debería haber en el 10%, 20% y 30% del vocabulario. Verifique respecto de los valores reales. Utilice esta aproximación para podar el vocabulario en los mismos porcentajes e indique qué porcentaje de la poda coincide con palabras vacías. Extraiga las palabras podadas que no son stopwords y verifique si, a su criterio, pueden ser importantes para la recuperación

In [54]:
import re

def sacar_acentos(texto):
    texto = re.sub(r"[áàäâã]", "a", texto)
    texto = re.sub(r"[éèëê]", "e", texto)
    texto = re.sub(r"[íìïî]", "i", texto)
    texto = re.sub(r"[óòöôõ]", "o", texto)
    texto = re.sub(r"[úùüû]", "u", texto)
    return texto


def procesar(texto):
    datos = {}
    texto = texto.lower()  # Convertir a minusculas
    texto = sacar_acentos(texto)
    texto = re.sub(r"[^a-zñ\s]", "", texto)  # Eliminar caracteres especiales
    tokens = texto.split()  # Tokenizar (En este caso separo por espacios)
    
    for token in tokens:
        datos[token] = datos.get(token, 0) + 1  
    
    return datos

with open("quijote.txt", "r", encoding="utf-8") as f:
    texto = f.read()
    frecuencias = procesar(texto.strip())
    datos = dict(sorted(frecuencias.items(), key=lambda item: item[1], reverse=True))
    
print(datos)


{'que': 21616, 'de': 18495, 'y': 18271, 'la': 10492, 'a': 9934, 'el': 9547, 'en': 8284, 'no': 6356, 'se': 5139, 'los': 4769, 'con': 4275, 'por': 3945, 'lo': 3492, 'las': 3486, 'le': 3420, 'su': 3388, 'don': 2718, 'del': 2668, 'como': 2554, 'me': 2345, 'si': 2314, 'mas': 2295, 'quijote': 2245, 'mi': 2207, 'sancho': 2174, 'es': 2145, 'yo': 2077, 'un': 1943, 'dijo': 1808, 'al': 1757, 'para': 1465, 'porque': 1400, 'ni': 1377, 'una': 1334, 'tan': 1245, 'o': 1217, 'todo': 1181, 'esta': 1165, 'sin': 1158, 'señor': 1065, 'asi': 1065, 'respondio': 1063, 'ser': 1059, 'bien': 1052, 'ha': 1052, 'sus': 1051, 'habia': 1037, 'pero': 1015, 'merced': 900, 'quien': 894, 'esto': 886, 'pues': 865, 'vuestra': 852, 'todos': 818, 'este': 818, 'ya': 785, 'donde': 784, 'cuando': 778, 'era': 757, 'cual': 749, 'te': 726, 'sino': 694, 'dos': 685, 'caballero': 677, 'fue': 655, 'ella': 604, 'tu': 599, 'decir': 578, 'he': 539, 'muy': 535, 'hacer': 535, 'aquel': 534, 'dios': 531, 'aunque': 531, 'señora': 518, 'otra':

In [ ]:
import numpy as np
import nltk
from nltk.corpus import stopwords

def podar_vocabulario(datos, cantidad_a_podar):
    total_frecuencias = 0
    palabras_a_conservar = []
    
    for palabra, frecuencia in datos.items():
        total_frecuencias += frecuencia
        if total_frecuencias >= cantidad_a_podar:
            break
        palabras_a_conservar.append((palabra, frecuencia))
        
    vocabulario_podado = dict(palabras_a_conservar)
    
    return vocabulario_podado

def calcular_porcentaje_palabras_vocabulario(datos, porcentaje):
    total_palabras = len(datos)
    limite = int(total_palabras * porcentaje / 100)  # Limite de palabras a tomar
    palabras = list(datos.items())[:limite]
    frecuencia_total = sum(frecuencia for _, frecuencia in palabras)
    return frecuencia_total

def calcular_palabras_zipf(datos, porcentaje):
    lista_datos = list(datos.items())
    total_palabras = len(lista_datos)
    limite = int(total_palabras * porcentaje / 100)
    
    frecuencias = np.array([freq for _, freq in lista_datos[:limite]])
    rangos = np.arange(1, limite + 1)
    
    # Ajuste de Zipf 
    coeficientes = np.polyfit(np.log(rangos), np.log(frecuencias), 1)
    C, b = np.exp(coeficientes[1]), coeficientes[0]
    
    frecuencias_estimadas = C * rangos ** b
    
    return int(np.sum(frecuencias_estimadas))

nltk.download('stopwords')
stop_words = set(stopwords.words("spanish"))
porcentajes = [10, 20, 30]

for p in porcentajes:
    reales = calcular_porcentaje_palabras_vocabulario(datos, p)
    estimadas_zipf = calcular_palabras_zipf(datos, p)
    podadas_reales = podar_vocabulario(datos, reales)   
    podadas_zipf = podar_vocabulario(datos, estimadas_zipf) 
       
    stopwords_podadas_reales = [w for w in podadas_reales if w in stop_words]   
    stopwords_podadas_zipf = [w for w in podadas_zipf if w in stop_words]
    no_stopwords_podadas_reales = [w for w in podadas_reales if w not in stop_words]   
    no_stopwords_podadas_zipf = [w for w in podadas_zipf if w not in stop_words]
    
    print(f"\nPoda del {p}%:")
    print(f"Palabras reales: {reales}")
    print(f"Palabras estimadas segun Zipf: {estimadas_zipf}")
    print(f"Total palabras eliminadas reales: {len(podadas_reales)}")
    print(f"Total palabras eliminadas segun Zipf: {len(podadas_zipf)}")
    print(f"Stopwords eliminadas (Reales): {len(stopwords_podadas_reales)} ({len(stopwords_podadas_reales) / len(podadas_reales) * 100:.2f}%)")
    print(f"Stopwords eliminadas (Zipf): {len(stopwords_podadas_zipf)} ({len(stopwords_podadas_zipf) / len(podadas_zipf) * 100:.2f}%)")
    print(f"Ejemplo de palabras importantes eliminadas (Reales): {no_stopwords_podadas_reales[:10]}")
    print(f"Ejemplo de palabras importantes eliminadas (Zipf): {no_stopwords_podadas_zipf[:10]}")


386619

Poda del 10%:
Palabras reales: 334926
Palabras estimadas segun Zipf: 396282
Total palabras eliminadas reales: 2313
Total palabras eliminadas segun Zipf: 23141
Stopwords eliminadas (Reales): 157 (6.79%)
Stopwords eliminadas (Zipf): 203 (0.88%)
Ejemplo de palabras importantes eliminadas (Reales): ['don', 'si', 'mas', 'quijote', 'sancho', 'dijo', 'tan', 'señor', 'asi', 'respondio']
Ejemplo de palabras importantes eliminadas (Zipf): ['don', 'si', 'mas', 'quijote', 'sancho', 'dijo', 'tan', 'señor', 'asi', 'respondio']
386619

Poda del 20%:
Palabras reales: 354456
Palabras estimadas segun Zipf: 478412
Total palabras eliminadas reales: 4627
Total palabras eliminadas segun Zipf: 23141
Stopwords eliminadas (Reales): 174 (3.76%)
Stopwords eliminadas (Zipf): 203 (0.88%)
Ejemplo de palabras importantes eliminadas (Reales): ['don', 'si', 'mas', 'quijote', 'sancho', 'dijo', 'tan', 'señor', 'asi', 'respondio']
Ejemplo de palabras importantes eliminadas (Zipf): ['don', 'si', 'mas', 'quijote', 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
